In [ ]:
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import os
from PIL import Image
import imageio

In [ ]:
def png2gif(file_name):
    images = []
    filenames=sorted((fn for fn in os.listdir(file_name) if fn.endswith('.png')))
    for filename in filenames:
        images.append(imageio.imread(file_name+'/'+filename))
    imageio.mimsave('result.gif', images,duration=0.5)

In [ ]:
def move_latent_and_save(latent_vector, direction_file, coeffs, Gs_network, Gs_syn_kwargs):
    direction = np.load('latent_directions/' + direction_file)
    os.makedirs('results/'+direction_file.split('.')[0], exist_ok=True)
    '''latent_vector是人脸潜编码，direction是人脸调整方向，coeffs是变化步幅的向量，generator是生成器'''
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[0][:8] = (latent_vector[0] + coeff*direction)[:8]
        images = Gs_network.components.synthesis.run(new_latent_vector, **Gs_syn_kwargs)
        result = PIL.Image.fromarray(images[0], 'RGB')
        result.save('results/'+direction_file.split('.')[0]+'/'+str(i).zfill(3)+'.png')

In [ ]:
def main():

    # 在这儿选择生成器
    tflib.init_tf()
    with open('networks/stylegan2-ffhq-config-f.pkl', "rb") as f:
        generator_network, discriminator_network, Gs_network = pickle.load(f)

    # 这是一些配置参数，不要动它
    w_avg = Gs_network.get_var('dlatent_avg')
    noise_vars = [var for name, var in Gs_network.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = 1
    truncation_psi = 0.5
    
    nw = np.load('./latent_representations/3.npy')[np.newaxis, :]
    direction_file = 'smile.npy'  # 从上面的编辑向量中选择一个

    # 在这儿选择调整的大小，向量里面的值表示调整幅度，可以自行编辑，对于每个值都会生成一张图片并保存。
    coeffs = [-2., -1.5, -0.75, 0., 0.75, 1.5, 2.]

    # 开始调整并保存图片
    move_latent_and_save(nw, direction_file, coeffs, Gs_network, Gs_syn_kwargs)

In [ ]:
main()

In [ ]:
png2gif('./results/smile/')

In [1]:
import cv2

In [22]:
img = cv2.imread('./aligned_images/gls.png')

In [23]:
img = cv2.resize(img,(1024,1024))

In [24]:
cv2.imwrite('./aligned_images/gls.jpg', img)

True